Testing surface oil beaching using [Parcels](https://oceanparcels.org)

In [1]:
import sys
import os
import math
import xarray as xr
import matplotlib.pyplot as plt
from pathlib import Path
from pprint import pprint
from parcels import VectorField, Variable #AdvectionRK4
from parcels import FieldSet, plotTrajectoriesFile, Variable, ScipyParticle, Field, ParcelsRandom
import numpy as np
from datetime import timedelta
sys.path.append('/ocean/rlovindeer/Atlantis/ssam_oceanparcels/Parcels_Utils/particle_tracking/parcels/')
from util.seed_particles import get_particles, get_release_times
# from util.parse_wildcards import parse_wildcards

INFO: Compiled ParcelsRandom ==> /tmp/parcels-2926/libparcels_random_f9a26b53-3bdf-48b8-8c21-d42eaf5be75d.so


In [2]:
# Spill release times
release_start_time = '2019-01-12'  ## winter start on December, Summer Jul - Aug  ## ask Susan about when to do simulation
release_end_time = '2019-01-13'
release_YYYY = '2019-01'

Select the location of interest, which is specified by a shapefile.

In [3]:
file_id = int(input( ))
scenario = {1 : "5b_Turn_Point_Diluted_bitumen",
            2 : "6a_VancouverHarbour_BunkerC",
            3 : "7a_JStrait_BunkerC",
            4 : "4a_ActivePass_Diesel",
            5 : "SandHeads"}
print("\nScenario running  :", scenario[file_id], sep = " ")


Scenario running  : 5b_Turn_Point_Diluted_bitumen


In [4]:
#Kernels

def AdvectionRK4(particle, fieldset, time):
    """Advection of particles using fourth-order Runge-Kutta integration.
    Function needs to be converted to Kernel object before execution"""
    if particle.beached == 0:
        (u1, v1) = fieldset.UV[particle]
        lon1, lat1 = (particle.lon + u1*.5*particle.dt, particle.lat + v1*.5*particle.dt)
        (u2, v2) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat1, lon1, particle]
        lon2, lat2 = (particle.lon + u2*.5*particle.dt, particle.lat + v2*.5*particle.dt)
        (u3, v3) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat2, lon2, particle]
        lon3, lat3 = (particle.lon + u3*particle.dt, particle.lat + v3*particle.dt)
        (u4, v4) = fieldset.UV[time + particle.dt, particle.depth, lat3, lon3, particle]
        particle.lon += (u1 + 2*u2 + 2*u3 + u4) / 6. * particle.dt
        particle.lat += (v1 + 2*v2 + 2*v3 + v4) / 6. * particle.dt
    
def WindAdvectionRK4(particle, fieldset, time):
    """Advection of particles using fourth-order Runge-Kutta integration.
     Function needs to be converted to Kernel object before execution"""
    if particle.beached == 0:
        wp = fieldset.wind_percentage ## this need to be add to the fieldset
        
        if wp > 0:

            (u1, v1) = fieldset.UVwind[time, particle.depth, particle.lat, particle.lon]
            u1 = u1 * wp
            v1 = v1 * wp
            lon1, lat1 = (particle.lon + u1*.5*particle.dt, particle.lat + v1*.5*particle.dt)
            
            (u2, v2) = fieldset.UVwind[time + .5 * particle.dt, particle.depth, lat1, lon1]
            u2 = u2 * wp
            v2 = v2 * wp
            lon2, lat2 = (particle.lon + u2*.5*particle.dt, particle.lat + v2*.5*particle.dt)
            
            (u3, v3) = fieldset.UVwind[time + .5 * particle.dt, particle.depth, lat2, lon2]
            u3 = u3 * wp
            v3 = v3 * wp
            lon3, lat3 = (particle.lon + u3*particle.dt, particle.lat + v3*particle.dt)
            
            (u4, v4) = fieldset.UVwind[time + particle.dt, particle.depth, lat3, lon3]
            u4 = u4 * wp
            v4 = v4 * wp            
            
            u_wind  = (u1 + 2*u2 + 2*u3 + u4) / 6. * particle.dt
            v_wind  = (v1 + 2*v2 + 2*v3 + v4) / 6. * particle.dt
            
            particle.lon += (u1 + 2*u2 + 2*u3 + u4) / 6. * particle.dt
            particle.lat += (v1 + 2*v2 + 2*v3 + v4) / 6. * particle.dt
            
            particle.beached = 2

def beaching(particle, fieldset, time):
    
    if particle.beached == 2:
        
        # Parameters
        deg2m = 111000     # [m/deg]
        D = 50   # maximum beaching distance [m]
        
        # Grounding probability over radius D
        offset_lon = random.uniform(-1., 1.) * D / deg2m
        offset_lat = random.uniform(-1., 1.) * D / deg2m

        # Evaluate if chosen point is over land
        (u, v) = fieldset.UV[time, particle.depth, particle.lat + offset_lat, particle.lon + offset_lon]
        if u == 0 and v == 0: 
            particle.beached = 1
        else:
            particle.beached = 0

def DeleteParticle(particle, fieldset, time):
    particle.delete()

def DecayParticle(particle, fieldset, time):
    dt = particle.dt
    field_decay_value = fieldset.decay
    decay = math.exp(0 * dt/field_decay_value) # -0.099 for diesel, 0 for other oils
    particle.decay_value = particle.decay_value * decay

In [5]:
# Data Paths
currents = Path('/ocean/rlovindeer/Atlantis/Physics/Raw_Transport_Data/')
winds = Path('/ocean/rlovindeer/Atlantis/Physics/Wind/')
sea_grid = Path('/ocean/rlovindeer/Atlantis/Physics/Grids/ubcSSnBathymetryV17-02_a29d_efc9_4047.nc')
air_grid = Path('/ocean/rlovindeer/Atlantis/Physics/Grids/ubcSSaAtmosphereGridV1_0f03_6268_df4b.nc')

In [6]:
# Salish Sea NEMO Model Grid, Geo-location and Bathymetry, v17-02

# Currents
# u_data_path = currents + '2018-01*URaw_variables.nc'
# v_data_path = currents + '2018-01*VRaw_variables.nc'
# u_current = parse_wildcards(u_data_path, 'u')
# v_current = parse_wildcards(v_data_path, 'v')

u_current = sorted([p for p in currents.glob(str(release_YYYY) + '*URaw_variables.nc')])
v_current = sorted([p for p in currents.glob(str(release_YYYY) + '*VRaw_variables.nc')])

filenames = {
    'U': {'lon': sea_grid,'lat': sea_grid,'data': u_current},
    'V': {'lon': sea_grid,'lat': sea_grid,'data': v_current}
            }

variables = {'U': 'uVelocity','V': 'vVelocity'}
dimensions = {'lon': 'longitude', 'lat': 'latitude', 'time': 'time'}
print('creating from_nemo')
fieldset = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
print('creating from_nemo done')

fieldset.add_constant('decay', 1.0 * 3600.0)
print('add_constant decay')

creating from_nemo
creating from_nemo done
add_constant decay


In [7]:
# HRDPS, Salish Sea, Atmospheric Forcing Grid, Geo-location, v1"

wind_paths = sorted([p for p in winds.glob(str(release_YYYY) + '*Wind_variables.nc')])
wind_filenames = {'lon': os.fspath(air_grid),'lat': os.fspath(air_grid),'data': wind_paths}
wind_dimensions = {'lon': 'longitude', 'lat': 'latitude', 'time': 'time'}

pprint(wind_filenames)

{'data': [PosixPath('/ocean/rlovindeer/Atlantis/Physics/Wind/2019-01-01_Wind_variables.nc'),
          PosixPath('/ocean/rlovindeer/Atlantis/Physics/Wind/2019-01-07_Wind_variables.nc'),
          PosixPath('/ocean/rlovindeer/Atlantis/Physics/Wind/2019-01-13_Wind_variables.nc'),
          PosixPath('/ocean/rlovindeer/Atlantis/Physics/Wind/2019-01-19_Wind_variables.nc'),
          PosixPath('/ocean/rlovindeer/Atlantis/Physics/Wind/2019-01-25_Wind_variables.nc'),
          PosixPath('/ocean/rlovindeer/Atlantis/Physics/Wind/2019-01-31_Wind_variables.nc')],
 'lat': '/ocean/rlovindeer/Atlantis/Physics/Grids/ubcSSaAtmosphereGridV1_0f03_6268_df4b.nc',
 'lon': '/ocean/rlovindeer/Atlantis/Physics/Grids/ubcSSaAtmosphereGridV1_0f03_6268_df4b.nc'}


In [8]:
Uwind_field = Field.from_netcdf(wind_filenames, ('U_wind', 'u_wind'),
                                     wind_dimensions,
                                     fieldtype='U',
                                     allow_time_extrapolation=True,
                                     transpose=False,
                                     deferred_load=False)
Vwind_field = Field.from_netcdf(wind_filenames, ('V_wind', 'v_wind'),
                                     wind_dimensions,
                                     fieldtype='V',
                                     allow_time_extrapolation=True,
                                     transpose=False,
                                     deferred_load=False)

print('wind data loaded')

wind data loaded


In [9]:
# change longitude for the wind field
Uwind_field.grid.lon = Uwind_field.grid.lon - 360
Vwind_field.grid.lon = Vwind_field.grid.lon - 360

[x_min, x_max, y_min, y_max] = Uwind_field.grid.lonlat_minmax

Uwind_field.grid.lonlat_minmax = [x_min - 360, x_max - 360, y_min, y_max]
Vwind_field.grid.lonlat_minmax = [x_min - 360, x_max - 360, y_min, y_max]

## adding the wind field to the fieldset object
fieldset.add_field(Uwind_field)
fieldset.add_field(Vwind_field)
wind_field = VectorField('UVwind', Uwind_field,  Vwind_field)
fieldset.add_vector_field(wind_field)

In [10]:
# wind_percentage
# We need to do a sensitivity analysis of the percetage of wind to be used here
wind_percentage = 3 #should this be 3?
fieldset.add_constant('wind_percentage', wind_percentage/100.0)

In [11]:
class MyParticle(ScipyParticle):
    initial_time = -100
    decay_value = Variable('decay_value', dtype=np.float32, initial=1.0)
    beached = Variable('beached', dtype=np.int32, initial=0.)
    age = Variable('age', dtype=np.int32, initial=0.)

# Particle Features
num_particles_per_day = 100
feature_release_index = 0
input_shapefile_name = "/ocean/rlovindeer/Atlantis/ssam_oceanparcels/SalishSea/Shape_Scenarios/" + scenario[file_id] + ".shp"
release_depth = -0.1
release_start_time = np.datetime64(release_start_time)
release_end_time = np.datetime64(release_end_time)
time_origin = fieldset.U.grid.time_origin.time_origin

print('setting up particles')

[release_times, p, num_particles] = get_release_times(time_origin, num_particles_per_day, release_start_time, release_end_time)
pset = get_particles(fieldset, num_particles, input_shapefile_name, MyParticle, feature_release_index, release_times, release_depth)

#print(pset)

# Building the kernels
decay_kernel = pset.Kernel(DecayParticle)
beaching_kernel = pset.Kernel(beaching)
ForcingWind_kernel = pset.Kernel(WindAdvectionRK4)
Advection_kernel = pset.Kernel(AdvectionRK4)

# Adding to the main kernel
my_kernel = Advection_kernel + decay_kernel + ForcingWind_kernel + beaching_kernel

output_file_name = scenario[file_id] + str(release_start_time) +  '_OP_D50_wp3.nc'
print(output_file_name)

setting up particles
689400.0
100
100
Using shapefile /ocean/rlovindeer/Atlantis/ssam_oceanparcels/SalishSea/Shape_Scenarios/5b_Turn_Point_Diluted_bitumen.shp
Number of sites :  1
Up to 0 of 1
80
100
Sucessfully seeded particles

num_attempts = 2
[-123.29342181743614, -123.29308906571126, -123.29331391769186, -123.29342967807801, -123.29367213484532, -123.29322765621787, -123.29364427002147, -123.29273589844243, -123.29375256140635, -123.29293599436784, -123.2934616546045, -123.29338335532181, -123.29266825116741, -123.2943451858446, -123.2928542047529, -123.2929631309421, -123.29277942014751, -123.29292112731557, -123.29359648571949, -123.29295838609143, -123.29428289559226, -123.29323960240134, -123.29263010660989, -123.2934757478005, -123.29369012056402, -123.2939903332198, -123.29297097706512, -123.29360714377957, -123.29338257654626, -123.29448186484314, -123.29328417344985, -123.29329525299856, -123.29328557645024, -123.29263194828697, -123.29315580384579, -123.29380042864285, -1

In [12]:
try:
    os.system('rm ' + output_file_name)
except:
    pass

print('executing particle kernel')

executing particle kernel


rm: cannot remove '5b_Turn_Point_Diluted_bitumen2019-01-12_OP_D50_wp3.nc': No such file or directory


In [13]:
## Output properties
output_file = pset.ParticleFile(name= output_file_name, outputdt = timedelta(minutes = 60))
pset.execute(my_kernel,                 # the kernel (which defines how particles move)
             runtime=timedelta(hours = 24*7),   # the total length of the run
             dt = timedelta(minutes = 20),      # the timestep of the kernel
             output_file = output_file)  # the file name and the time step of the outputs
output_file.close()

plotTrajectoriesFile(output_file_name);

print('particle trajectories completed')

INFO: Temporary output files are stored in out-DLRCUQYY.
INFO: You can use "parcels_convert_npydir_to_netcdf out-DLRCUQYY" to convert these to a NetCDF file during the run.
N/A% (0 of 604800.0) |                   | Elapsed Time: 0:00:00 ETA:  --:--:--
  4% (25200.0 of 604800.0) |             | Elapsed Time: 0:00:02 ETA:   0:00:57
  4% (28800.0 of 604800.0) |             | Elapsed Time: 0:00:07 ETA:   0:14:26
  5% (32400.0 of 604800.0) |             | Elapsed Time: 0:00:13 ETA:   0:13:44
  5% (36000.0 of 604800.0) |             | Elapsed Time: 0:00:18 ETA:   0:14:35
  6% (39600.0 of 604800.0) |             | Elapsed Time: 0:00:24 ETA:   0:15:15
  7% (43200.0 of 604800.0) |             | Elapsed Time: 0:00:30 ETA:   0:16:15
  7% (44400.0 of 604800.0) |             | Elapsed Time: 0:00:32 ETA:   0:17:36
  7% (45000.0 of 604800.0) |             | Elapsed Time: 0:00:34 ETA:   0:19:41
  7% (46800.0 of 604800.0) |#            | Elapsed Time: 0:00:38 ETA:   0:21:44
  8% (50400.0 of 604800.0) 

Correct cell not found after 1000000 iterations
Correct cell not found after 1000000 iterations


OutOfBoundsError: 0
Particle P[16](lon=-124.892400, lat=48.629780, depth=0.000000, decay_value=1.000000, beached=0.000000, age=0.000000, time=1207200.000000)
Time: 2019-01-17T23:50:00.000000000,	timestep dt: 600.000000
Field sampled at (-124.895716, 48.631422, 0.000000)

In [1]:
# Creating a netcdf file from a partial Ocean Parcels run that was aborted because particle moved out of bounds

from argparse import ArgumentParser
from glob import glob
from os import path

import numpy as np

# == here those classes need to be impported to parse available ParticleFile classes and create the type from its name == #
from parcels import ParticleFile

def convert_npydir_to_netcdf(tempwritedir_base, delete_tempfiles=False, pfile_class=None):
    """Convert npy files in tempwritedir to a NetCDF file
    :param tempwritedir_base: directory where the directories for temporary npy files
            are stored (can be obtained from ParticleFile.tempwritedir_base attribute)
    """

    tempwritedir = sorted(glob(path.join("%s" % tempwritedir_base, "*")),
                          key=lambda x: int(path.basename(x)))[0]
    pyset_file = path.join(tempwritedir, 'pset_info.npy')
    if not path.isdir(tempwritedir):
        raise ValueError('Output directory "%s" does not exist' % tempwritedir)
    if not path.isfile(pyset_file):
        raise ValueError('Output directory "%s" does not contain a pset_info.npy file' % tempwritedir)

    pset_info = np.load(pyset_file, allow_pickle=True).item()
    pfconstructor = ParticleFile if pfile_class is None else pfile_class
    pfile = pfconstructor(None, None, pset_info=pset_info, tempwritedir=tempwritedir_base, convert_at_end=False)
    pfile.close(delete_tempfiles)


def main(tempwritedir_base=None, delete_tempfiles=False):
    if tempwritedir_base is None:
        p = ArgumentParser(description="""Script to convert temporary npy output files to NetCDF""")
        p.add_argument('tempwritedir', help='Name of directory where temporary npy files are stored '
                                            '(not including numbered subdirectories)')
        p.add_argument('-d', '--delete_tempfiles', default=False,
                       help='Flag to delete temporary files at end of call (default False)')
        p.add_argument('-c', '--pfclass_name', default='ParticleFileSOA',
                       help='Class name of the stored particle file (default ParticleFileSOA)')
        args = p.parse_args()
        tempwritedir_base = args.tempwritedir
        pfclass = ParticleFile
        if hasattr(args, 'delete_tempfiles'):
            delete_tempfiles = args.delete_tempfiles
        if hasattr(args, 'pfclass_name'):
            try:
                pfclass = locals()[args.pfclass_name]
            except:
                pfclass = ParticleFile

    convert_npydir_to_netcdf(tempwritedir_base, delete_tempfiles, pfile_class=pfclass)


if __name__ == "__main__":
    main()

INFO: Compiled ParcelsRandom ==> /tmp/parcels-2926/libparcels_random_715a372a-29cd-481a-bc67-8522aafbb525.so
usage: ipykernel_launcher.py [-h] [-d DELETE_TEMPFILES] [-c PFCLASS_NAME]
                             tempwritedir
ipykernel_launcher.py: error: the following arguments are required: tempwritedir


SystemExit: 2

/home/rlovindeer/conda_envs/parcels-dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
